In [ ]:
from pprint import pprint
from copy import deepcopy

import requests
import grequests

from pymlaas.util import request_helpers

In [ ]:
# url_pymlaas_dl = 'http://127.0.0.1:5000/parse_driver_license'
url_pymlaas_dl = 'http://pyml.taxi.dev.yandex.net/parse_driver_license'

In [ ]:
prod = '8b3818d4-259d-46e6-89be-cd266f5ae81f'
test = '49f1d123-e803-4fda-b727-62ed0e3c7a4a'

### Single response checks

In [ ]:
photo_name_to_test = 'dl_rotated_270.jpg'

In [ ]:
front_binary_string = "front_binary=@./example_photos/{}".format(
    photo_name_to_test
)
resp = !curl $url_pymlaas_dl \
        -F  $front_binary_string \
        -F "country=rus" \
        -H "Host: pyml.taxi.yandex.net" -v
try:
    pprint(json.loads(resp[-1]))
except:
    pprint(resp[-1])

In [ ]:
print 'first_name:', json.loads(resp[-1])['first_name']['value']
print 'last_name:', json.loads(resp[-1])['last_name']['value']
print 'middle_name:', json.loads(resp[-1])['middle_name']['value']
print 'number:', json.loads(resp[-1])['number']['value']
print 'series:', json.loads(resp[-1])['series']['value']
print 'due_date:', json.loads(resp[-1])['due_date']['value']
print 'issue_date:', json.loads(resp[-1])['issue_date']['value']
print 'birth_date:', json.loads(resp[-1])['birth_date']['value']

In [ ]:
files = {}
with open('./example_photos/' + photo_name_to_test, 'rb') as f:
    photo_binary = f.read()

files['file'] = photo_binary
files['langName'] = 'eng,rus'
files['apikey'] = test
files['meta'] = json.dumps({"StrategyName": "FullOcrVU"})
files['rotate'] = 'auto'

req_success = requests.request(
    method='post',
    url='https://api-translate.ocr.yandex.net:443/recognize',
    files=files,
    timeout=3
)

In [ ]:
resp_text = req_success.json()

In [ ]:
# resp_text

In [ ]:
for block in resp_text['data']['blocks']:
    for box in block['boxes']:
        for language in box['languages']:
            for text in language['texts']:
                for word in text['words']:
                    print word['word']

### Response consistency checks

In [ ]:
for i in range(1):
    resp = !curl $url_pymlaas_dl \
            -F "front_binary=@./example_photos/blank.jpg" \
            -F "country=rus" \
            -H "Host: pyml.taxi.yandex.net" -v
    try:
        pprint(json.loads(resp[-1]))
    except:
        pprint(resp[-1])

In [ ]:
files = {}
with open('./example_photos/blank.jpg', 'rb') as f:
    photo_binary = f.read()

files['file'] = photo_binary
files['langName'] = 'eng,rus'
files['apikey'] = test
files['meta'] = json.dumps({"StrategyName": "FullOcrVU"})
files_no_api = deepcopy(files)

req_success = grequests.request(
    method='post',
    url='https://api-translate.ocr.yandex.net:443/recognize',
    files=files,
    timeout=3
)

req_timeout = grequests.request(
    method='post',
    url='https://api-translate.ocr.yandex.net:443/recognize',
    files=files,
    timeout=0.1
)

files_no_api.pop('apikey')
req_no_api = grequests.request(
    method='post',
    url='https://api-translate.ocr.yandex.net:443/recognize',
    files=files_no_api,
    timeout=3
)

glist = grequests.map([req_timeout]*2 + [req_no_api]*2 + [req_success]*2,
                      exception_handler=request_helpers.default_exception_handler)
# glist = grequests.map([req_success]*2, exception_handler=exception_handler)

In [ ]:
!curl -kD - "https://api-translate.ocr.yandex.net:443/recognize" \
-F "file=@./example_photos/sergey_larin_dl_bright.jpg" \
-F "apikey=49f1d123-e803-4fda-b727-62ed0e3c7a4a" \
-F "langName=eng,rus" \
-H "Expect:" \
-F "meta={\"StrategyName\":\"FullOcrVU\"}" \
-F "rotate=auto"